### 수어 형식으로 형태소 변환하기 위한 Class정의

In [1]:
class Morph:
    MORPH = {   # 명사
                'NNG': '명사', 'NNP': '명사', 'NNB': '명사','NF': '명사', 'NP': '명사',
                # 용언
                'VV': '용언', 'VA': '용언', 'VX': '용언', 'VCP': '용언','VCN': '용언', 'NV': '용언', 'XSV': '용언',
                # 관형사
                'MM': '관형사',
                # 부사
                'MAG': '부사', 'MAJ': '부사',
                # 감탄사
                'IC': '감탄사',
                # 조사/
                'JKS': '조사','JKC': '조사', 'JKG': '조사', 'JKO': '조사', 'JKB': '조사', 'JKV': '조사',
                'JKQ': '조사', 'JX': '조사', 'JC': '조사',
                # 어미
                'EP': '어미','EF': '어미','EC': '어미','ETN': '어미','ETM': '어미',
                # 접사
                'XPN': '접사','XSN': '접사','XSA': '접사', 'XR': '접사',
                # 부호
                'SF': '기호','SP': '기호','SS': '기호','SE': '기호','SO': '기호','SW': '기호',
                # 숫자, 수사
                'SN': '숫자', 'NR': '수사',
                # 영어
                'SL': '영어',
                # 한자 제외
                'SH': '없음',
                # 분석 불능 범주 제외
                'NA': '없음'
    }

    # 사용하는 관형사
    USE_DETER = ['저', '맨', '별', '무슨', '헌', '온', '온갖', '그', '동', '오랜', '모든', '그런', '양', '딴', '각', '어떤', '이런',
                 '몹쓸', '약', '옛', '어느', '한', '이러한']

    # 사용하는 조사
    USE_POST = ['랑', '에서', '서', '더러', '보다', '에게', '의', '로', '이라고', '에', '처럼', '께', '으로', '한테', '라고',  '게', '로서', # 격조사
                 '로써', '으로서', '으로써', '만치', '만큼', '보고', '보다', '서부터', '에서부터', '시여', '이시여', '아',# 격조사
                '이나', '든지', '부터', '도', '커녕', '마다', '밖에', '뿐', '만', '까지', '라도', '이라도', '라든지', '거나', '나', '나마', # 보조사
                '이나마', '는', '다가', '대로', '든가', '들', '따라', '라야', '란', '이란','로부터', '마는', '마저', '밖에', '부터', '뿐', # 보조사
                '야말로', '이야말로', '에는', '에다', '에다가', '은',  '이라야', '이야', '조차',# 보조사
                '과', '와', '이랑', '고', '이고', '며', '이며',# 접속조사
                '이니까', '이다', '야'  # 서술격 조사
                ]

    # 사용하는 어미
    USE_END = ['았', '었', 'ㅂ시다', '면서', '자', '지만', '던', '면', '러', '다오', 'ㅂ니까', '다가', '아라', '자',
               '고자', '예요', '고', '을까', '구나', '다면', '으면', '라고', '와', '느라고']

    # 수어사전에 없는 명사 있는 명사로 대체
    SPECIAL_NOUN = {'꺼': '것', '거': '것', '니': '너', '내': '나'}

    # 불규칙 어미
    SPECIAL_END = {'았': '끝', '었': '끝', '으면': '면', '다면': '면', '예요': '이다', '을까': 'ㅂ니까', '다면': '면'}

    # 서술격 조사 '이다'로 변경
    SPECIAL_POST = {'이': '이다'}

    # 불규칙 접사
    SPECIAL_AFFIX = {'스럽': '스럽다', '하': '하다', '어떠한': '어떤', '이러한': '이런', '그러한': '그런'}

    # 수어사전에 없는 부사 있는 부사로 대체
    SPECIAL_ADVERB = {'근데': '그런데', '각각': '각'}

    # 기호
    SPECIAL_MARK = {'?': 'ㅂ니까'}

In [2]:
class StopWord:
    def __init__(self):
        self.mp = Morph()

    def process_morph(self, morph, word):
        m = self.mp.MORPH[morph]
        FUN = {
            '명사': self.check_noun,  # 명사(noun)
            '용언': self.check_verb,  # 동사(verb)
            '관형사': self.default,  # 관형사(determinant)
            '부사': self.check_adverb,  # 부사(adverb)
            '감탄사': self.default,  # 감탄사(exclamation)
            '조사': self.ignore,  # 조사(post)
            '어미': self.check_end,  # 어미(end)
            '접사': self.check_affix,  #접사(affix)
            '숫자': self.check_number,  # 숫자(number),
            '수사': self.check_number,
            '기호': self.check_mark,  # 기호,
            '영어': self.ignore,
            '없음': self.ignore  # 무시해도 되는 품사들
        }
        fun = FUN[m]
        return fun(m, word)

    # 무조건 출력
    def default(self, morph, word):
        return 1, word, morph

    # 의미있는 명사 추출
    def check_noun(self, morph, word):
        if word in self.mp.SPECIAL_NOUN.keys():
            return 1, self.mp.SPECIAL_NOUN[word], morph
        else:
            return 1, word, morph

    # 의미있는 동사 추출
    def check_verb(self, morph, word):
        return 1, word+'다', morph

    def check_deter(self, morph, word):
        if word in self.mp.USE_DETER:
            return 1, word, morph
        else:
            return 0, '', ''

    # 의미있는 부사 추출
    def check_adverb(self, morph, word):
        if word in self.mp.SPECIAL_ADVERB.keys():
            return 1, self.mp.SPECIAL_ADVERB[word], morph
        else:
            return 1, word, morph

    # 의미있는 조사 추출
    def check_post(self, morph, word):
        if word in self.mp.USE_POST:
            if word in self.mp.SPECIAL_POST.keys():
                return 1, self.mp.SPECIAL_POST[word], morph
            else:
                return 1, word, morph
        else:
            return 0, '', ''

    # 의미있는 어미 추출
    def check_end(self, morph, word):
        if word in self.mp.USE_END:
            if word in self.mp.SPECIAL_END.keys():
                return 1, self.mp.SPECIAL_END[word], morph
            else:
                return 1, word, morph
        else:
            return 0, '', ''

    # 의미있는 접사 추출
    def check_affix(self, morph, word):
        if word in self.mp.SPECIAL_AFFIX.keys():
            return 1, self.mp.SPECIAL_AFFIX[word], morph
        else:
            return 1, word, morph

    # 숫자 표현(ex. 157 -> 100 50 7)
    def check_number(self, morph, word):
        if morph == '수사':
            return 1, word, morph
        else:
            number = int(word)
            # number의 자릿수
            cipher = len(word)
            text = ''
            for i in range(cipher):
                if word[i] == '0': continue
                text += str(int(word[i]) * (10 ** (cipher - 1 - i)))
                text += ' '
            return 1, text, morph

    # 기호 처리
    def check_mark(self, morph, word):
        if word in self.mp.SPECIAL_MARK.keys():
            return 1, self.mp.SPECIAL_MARK[word], morph
        else:
            return 0, '', ''

    # 무조건 제거
    def ignore(self, morph, word):
        return 0, '', ''

### komoran 형태소분석기 사용

In [3]:
from konlpy.tag import Komoran
komoran = Komoran(userdic='my_dict.txt')   # 내 사전 추가
# komoran = Komoran()


# 자막파일을 입력받아 수어에 사용되는 형태소를 재배치하는 함수를 가진 객체
class NLP:
    def __init__(self):
        self.komoran = komoran
        # self.komoran.set_user_dic(r"C:/Users/SSAFY/Desktop/ssafy공통PJT/pjt_backup/SSLIS/app/bin/dic.user")
        self.pr = StopWord()
        pass

    def splitLine(self, line):
        for i in range(len(line)):
            s = str(line[i])
#             print(s)
            s = s.replace('(', '')
            s = s.replace(')', '')
            s = s.replace("'", '')
            s = s.replace(" ", '')
#             print(s)
            s = s.split(',')
#             print(s)
            line[i] = s
#         print(line)
        return line

    # 수어에 사용되는 형태소를 재배치하는 함수
    def relocateMorpheme(self, subtitle_path):
        result=[]
        word_list=[]
        morph_list=[]

        line = self.komoran.pos(subtitle_path)
        line = self.splitLine(line)
        for w, m in line:
            r, word, morph = self.pr.process_morph(m, w)
            if r == 1:
                if (word == 'ㅂ니까') or (word == '하다') or (word == '끝'):
                    if len(result) == 0:
                        word_list.append(word)
                        morph_list.append(morph)
                    elif result[len(result) - 1][0] != word:
                        word_list.append(word)
                        morph_list.append(morph)
                else:
                    word_list.append(word)
                    morph_list.append(morph)
        result.append(word_list)
        result.append(morph_list)
        return result[0], result[1]




### STT

In [4]:
######## Google STT API : 타임스탬프포함
#f=open('script.txt')   #단어조합
f=open('sentence.txt')   #문장조합
text = f.read()
temp=list(text.split('\n'))
del temp[-1]


nlp=NLP()
ans=[]    #l을 담을 전체 리스트
l=[]   #각 문장의 단어, 형태소, 첫, 끝시간 포함 리스트
for data in temp:
    word, st, et = data.split(',')
    l=list(nlp.relocateMorpheme(word))
    l.append(float(et)-float(st))
#     l.append(round(float(et)-float(st),1))
#     l.append(st)
#     l.append(et)
    ans.append(l)

print(ans)  
# print(ans[1])  #[['왼손', '코딩', '의', '김해', '손', '이다'], ['명사', '명사', '명사', '명사', '명사', '용언'], '1.8', '3.3']
# print(ans[1][2])   #['왼손', '코딩', '의', '김해', '손', '이다']
# print(ans[1][0][0])  #왼손
# print(ans[1][0][0][0])  #왼

[[['안녕하세요'], ['명사'], 0.9], [['왼손', '코딩', '의', '김해', '손', '이다'], ['명사', '명사', '명사', '명사', '명사', '용언'], 1.4999999999999998], [['김', '왼손', '왼손', '코딩', '은', '비전공자', '위하다', '세상', '제일', '쉽다', '코딩', '교육', '목표', '하다', '채널', '이다'], ['명사', '명사', '명사', '명사', '명사', '명사', '용언', '명사', '명사', '용언', '명사', '명사', '명사', '용언', '명사', '용언'], 6.7], [['최근', '몇', '년', '간', '코딩', '되다'], ['명사', '관형사', '명사', '명사', '명사', '용언'], 2.1999999999999993], [['없다', '고', '당분간', '계속', '되다', '것', '같다'], ['용언', '어미', '부사', '명사', '용언', '명사', '용언'], 2.3000000000000007], [['이런', '트렌드', '따르다', '많다', '비전공자', '들', '코딩', '하다', '전하다', '고', '있다'], ['관형사', '명사', '용언', '용언', '명사', '접사', '명사', '용언', '용언', '어미', '용언'], 3.8999999999999986], [['많다', '인터넷', '사이트', '학원', '코딩', '싶다', '공부', '하다'], ['용언', '명사', '명사', '명사', '명사', '용언', '명사', '용언'], 3.3000000000000007], [['하지만', '대부분', '비전공자', '들다', '중간', '톡', '하다'], ['부사', '명사', '명사', '용언', '명사', '부사', '용언'], 3.6000000000000014], [['왜', '이렇다', '다', '들', '포기', '하다', '되다', '걸다'], ['부사', '용언', '명사', '

In [5]:
a=ans[0][2]
a

0.9

In [6]:
words_list=[]  #단어
gram_list=[]  #품사
for i in range(len(ans)):
    words_list.append(ans[i][0])
    gram_list.append(ans[i][1])
words_list

[['안녕하세요'],
 ['왼손', '코딩', '의', '김해', '손', '이다'],
 ['김',
  '왼손',
  '왼손',
  '코딩',
  '은',
  '비전공자',
  '위하다',
  '세상',
  '제일',
  '쉽다',
  '코딩',
  '교육',
  '목표',
  '하다',
  '채널',
  '이다'],
 ['최근', '몇', '년', '간', '코딩', '되다'],
 ['없다', '고', '당분간', '계속', '되다', '것', '같다'],
 ['이런', '트렌드', '따르다', '많다', '비전공자', '들', '코딩', '하다', '전하다', '고', '있다'],
 ['많다', '인터넷', '사이트', '학원', '코딩', '싶다', '공부', '하다'],
 ['하지만', '대부분', '비전공자', '들다', '중간', '톡', '하다'],
 ['왜', '이렇다', '다', '들', '포기', '하다', '되다', '걸다'],
 ['저', '컴퓨터공학', '출신', '이다'],
 ['어느', '전공', '되다', '적성', '맞다', '않다', '사람', '들', '있다'],
 ['제가', '바로', '그렇다', '끝'],
 ['세상', '코딩', '제일', '싫어하다', '끝'],
 ['이쪽',
  '분야',
  '이런',
  '사람',
  '들',
  '보통',
  '코딩',
  '보기',
  '잘',
  '하다',
  '고',
  '하다',
  '그래서',
  '저',
  '코딩',
  '어렵다',
  '포기',
  '하다',
  '비전공자',
  '마음',
  '잘',
  '알다',
  '고',
  '있다'],
 ['코딩', '어렵다'],
 ['특히', '코딩', '걸다', '처음', '접하다', '비전공자', '더욱', '어렵다'],
 ['지금', '이', '영상', '보다', '여러분', '코', '댕', '어렵다', '면', '잘', '찾아오다', '겁', '니다'],
 ['코딩',
  '교육',
  '유튜브',
  '채널',

### 숫자 처리 

### 유사한 단어 찾아 mapping

In [9]:
import pandas as pd
# df= pd.read_csv('all_preprocessed_KOR.csv')   #수화영상이 포함된 csv파일
# df= pd.read_csv('Preprocessed_crawl.csv', encoding = "cp949")   #수화영상이 포함된 csv파일
df= pd.read_csv('data_crawl.csv', encoding = "cp949")   #수화영상이 포함된 csv파일

In [10]:
df

,Unnamed: 0,kor,no,url,part,video_sec
0,0,0,1214,http://sldict.korean.go.kr/multimedia/multimed...,NaN,2.77
1,1,1,12573,http://sldict.korean.go.kr/multimedia/multimed...,NaN,2.69
2,2,10,10919,http://sldict.korean.go.kr/multimedia/multimed...,NaN,3.35
3,3,100,2939,http://sldict.korean.go.kr/multimedia/multimed...,NaN,3.02
4,4,1000,11983,http://sldict.korean.go.kr/multimedia/multimed...,NaN,2.69
...,...,...,...,...,...,...
26686,26686,힘주다,3971,http://sldict.korean.go.kr/multimedia/multimed...,NaN,11.25
26687,26687,힘차다,4585,http://sldict.korean.go.kr/multimedia/multimed...,NaN,3.40
26688,26688,힘차다,4885,http://sldict.korean.go.kr/multimedia/multimed...,NaN,3.40
26689,26689,힘차다,7465,http://sldict.korean.go.kr/multimedia/multimed...,NaN,3.40


In [11]:
#유사한 단어들의 url 리스트에 넣기 :원본
from difflib import get_close_matches
words = df['kor']
url=df['url']

link= []
for i in words_list:
    u=[]
    for j in i:
        word=j
        candidates=words
        n=1  #최대 문자 매칭 개수 
        cutoff=0.5  #유사도 하한
        close_matches = get_close_matches(word, candidates, n, cutoff)  #유사한것
        print(close_matches)
        for i in close_matches:
            ind=df.index[df['kor'] == i].tolist()
            indd=ind[0]  #동음의의어 있어서 일단 젤 앞에 있는 숫자 추출하도록
            href=url[indd]
            u.append(href)
    link.append(u)


['안녕하세요']
['왼손잡이']
['코딩']
['의']
['해']
['손']
['이다']
['김']
['왼손잡이']
['왼손잡이']
['코딩']
['은']
['전자']
['위하다']
['세상']
['제일']
['쉽다']
['코딩']
['교육']
['표']
['하다']
['풍채']
['이다']
['최근']
['몇']
['년']
['간']
['코딩']
['되다']
['없다']
['고']
['당분간']
['계속']
['되다']
['것']
['같다']
['이런']
[]
['따르다']
['많다']
['전자']
['들']
['코딩']
['하다']
['전하다']
['고']
['있다']
['많다']
['인터넷']
['사이트']
['학원']
['코딩']
['싶다']
['공부']
['하다']
['하지만']
['대부분']
['전자']
['들다']
['중간']
[]
['하다']
['왜']
['이렇다']
['다']
['들']
['포기']
['하다']
['되다']
['걸다']
['저']
['컴퓨터']
['출신']
['이다']
['어느']
['전공']
['되다']
['적성']
['맞다']
['않다']
['사람']
['들']
['있다']
['제']
['바로']
['그렇다']
['끝']
['세상']
['코딩']
['제일']
['싫어하다']
['끝']
['이쪽']
['분야']
['이런']
['사람']
['들']
['보통']
['코딩']
['보기']
['잘']
['하다']
['고']
['하다']
['그래서']
['저']
['코딩']
['어렵다']
['포기']
['하다']
['전자']
['마음']
['잘']
['알다']
['고']
['있다']
['코딩']
['어렵다']
['특히']
['코딩']
['걸다']
['처음']
['밀접하다']
['전자']
['더욱']
['어렵다']
['지금']
['이']
['동영상']
['보다']
['여러분']
['코']
[]
['어렵다']
['면']
['잘']
['찾아내다']
['겁']
['지니다']
['코딩']
['교육']
[]
['풍채']
['지']
['벌써']


In [12]:
len(ans), len(link)

(22, 22)

### 없는 단어일 경우 지화수어로 처리

### 영상병합

In [16]:
link[20]

['http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20160325/271794/MOV000273323_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191016/628031/MOV000236652_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191016/628031/MOV000236652_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191004/624330/MOV000236182_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20201211/785423/MOV000357080_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191016/628210/MOV000251513_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191015/627702/MOV000256909_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151121/213995/MOV000236269_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191015/627623/MOV000253055_700X466.mp4',
 'http://sldict.korean.go.kr

In [17]:
link[21]

['http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20190918/615442/MOV000240981_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20151217/229714/MOV000243342_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191016/628031/MOV000236652_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191004/624330/MOV000236182_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191014/627257/MOV000253623_700X466.mp4']

In [18]:
import os
import urllib.request
import glob
from moviepy.editor import VideoFileClip, concatenate_videoclips
import shutil
import re
import cv2
from moviepy.video.io.ffmpeg_tools import *
from moviepy.video.fx.crop import *
from moviepy.editor import *
import shutil

# clip = VideoFileClip(#파일경로)
# print(clip.duration)

def save_signlanguage_video(href,name, num):  #일치하는 수화영상을 크롭하여 저장
    output_location = f'word/word{num}/' + 'word_'+str(name)+ '.mp4'
    clip = VideoFileClip(href).subclip(1,3).resize((560,360))    #수화영상의 1~3초만 잘라냄 > 크기 조절

    new_clip = crop(clip, x1=70, y1=0, x2=490, y2=270)   #가슴까지 보이도록 crop
    new_clip.write_videofile(output_location)

    return output_location

def speedx(file_list,n):   # 수화영상들을 병합해서 배속
    clip_list = []
    for i in file_list:
        clip_list.append(VideoFileClip(i))
    final_clip = concatenate_videoclips(clip_list)
    print("final: ", final_clip.duration)
#     t=ans[n][2]/final_clip.duration
    t=round(final_clip.duration/ans[n][2],1)

    print("비율 : ", t)
    final_clip.speedx(t).write_videofile(f"sentence/sentence_{n}.mp4")
    

    
num = 0 #0   #줄 몇갠지
for i in link:
    n=0   #한 줄안에 들어갈 영상 몇갠지
    os.mkdir(f"word/word{num}")
    for j in i:
        href =j
        save_signlanguage_video(href, n, num)
        n+=1
    file_list = sorted(glob.glob(f'word/word{num}/*.mp4'),key=os.path.getctime)
    
    print(file_list)
    speedx(file_list, num)    
    num+=1

 




t:   3%|██▎                                                                   | 2/60 [00:00<00:04, 14.48it/s, now=None]

Moviepy - Building video word/word0/word_0.mp4.
Moviepy - Writing video word/word0/word_0.mp4



Moviepy - Done !
Moviepy - video ready word/word0/word_0.mp4
['word/word0\\word_0.mp4']


final:  2.0
비율 :  2.2
Moviepy - Building video sentence/sentence_0.mp4.
Moviepy - Writing video sentence/sentence_0.mp4



Moviepy - Done !
Moviepy - video ready sentence/sentence_0.mp4


...

Moviepy - Building video word/word21/word_4.mp4.
Moviepy - Writing video word/word21/word_4.mp4



Moviepy - Done !
Moviepy - video ready word/word21/word_4.mp4
['word/word21\\word_0.mp4', 'word/word21\\word_1.mp4', 'word/word21\\word_2.mp4', 'word/word21\\word_3.mp4', 'word/word21\\word_4.mp4']


t:   2%|█▍                                                                   | 3/143 [00:00<00:05, 26.33it/s, now=None]

final:  10.0
비율 :  4.2
Moviepy - Building video sentence/sentence_21.mp4.
Moviepy - Writing video sentence/sentence_21.mp4



Moviepy - Done !
Moviepy - video ready sentence/sentence_21.mp4


In [19]:
file_tot_list = sorted(glob.glob('sentence/*.mp4'), key=os.path.getctime)
clip_tot_list = []



for i in file_tot_list:
    clip_tot_list.append(VideoFileClip(i))
    
final_tot_clip = concatenate_videoclips(clip_tot_list)
final_tot_clip.write_videofile("total_demo1.mp4")

t:   0%|                                                                    | 3/5106 [00:00<03:03, 27.87it/s, now=None]

Moviepy - Building video total.mp4.
Moviepy - Writing video total.mp4



Moviepy - Done !
Moviepy - video ready total.mp4
